In [1]:
import pandas as pd 
import numpy as nmp
import pdfplumber
import warnings
warnings.filterwarnings("ignore")
import pdfplumber
from tabula import read_pdf
import re 
import os
directory = "C:\\Users\\User\\Desktop\\COMP2\\pdf"
import tabula

In [3]:
comps = os.listdir(directory)

In [4]:
file = "C:\\Users\\User\\Desktop\\COMP2\\pdf\\Медицинская оптика.pdf"

In [65]:
def parser(filepath, pg):
    L=[]
    pdf = pdfplumber.open(filepath)
    for i in range(pg):
        page = pdf.pages[i]
        try:
            table = page.extract_table()
            df = pd.DataFrame(table)
            if df.shape[0] > 1:
                df = df.drop([0,2,5,6,8], axis=1)
                for i in range(len(df)):
                    if df.iloc[i][4] != None:
                        df.iloc[i][3] = df.iloc[i][4]
                df = df.drop(4, axis=1)
                L.append(df)
                if "Всего" in table[-1]:
                    break
        except Exception:
            print("n", end = ",")
    df = pd.concat(L)
    df = df.reset_index(drop=True)
    df.columns = ["RAZDEL", "SLOVA", "VAZHNOST"]
    df = df.drop([0,1]).reset_index(drop=True)
    return df

In [257]:
DF=[]
NN=[]
for i in comps:
    fp = directory +"\\"+i
    try:
        print(i[:i.find(".")])
        df = parser(fp,14)
        df["COMP"] = i[:i.find(".")]
        DF.append(df)
    except Exception:
        NN.append(fp)
    

In [199]:
df = pd.concat(DF)

In [201]:
df.columns = ["Раздел","Слова","Важность","Компетенция"]

In [202]:
data = pd.read_excel("dataset.xlsx")

In [203]:
data.shape

(130, 32)

In [205]:
N = data.iloc[0][21]
for i in range(data.shape[0]):
    comp = data.iloc[i][1]
    for j in range(2, len(data.iloc[i])-1):
        tmp = {"Раздел":N,"Слова":N,"Важность":N,"Компетенция":comp}
        if type(data.iloc[i][j]) == int or type(data.iloc[i][j]) == float:
            continue
        if (type(data.iloc[i][j]) != int or type(data.iloc[i][j]) != float) and data.iloc[i][j] != N:
            tmp["Слова"] = data.iloc[i][j]
        if type(data.iloc[i][j+1]) == int or type(data.iloc[i][j+1]) == float:
            tmp["Важность"] = data.iloc[i][j+1]
        df = df.append(tmp, ignore_index=True)

In [208]:
dr = []
for i in range(len(df)):
    if type(df.iloc[i]["Слова"]) == float:
        df.loc[i-1]["Важность"] = df.iloc[i]["Слова"]
        dr.append(i)

In [209]:
df = df.drop(dr).reset_index(drop=True)

In [210]:
df = df[df["Слова"].notna()]

In [211]:
df = df.reset_index(drop=True)

In [212]:
df["Важность"] = df["Важность"].fillna(0)

In [196]:
v = int(df.iloc[0]["Важность"])
for i in range(1, df.shape[0]):
    if df.loc[i]["Важность"] != 0:
        v = df.loc[i]["Важность"]
    else:
        df.loc[i]["Важность"] = v

In [233]:
for i in range(df.shape[0]):
    df["Слова"].iloc[i]  = re.sub('[–\n\r."("")"©•,«»:;%$!@0-9$]', '', df["Слова"].iloc[i]).lower()

In [223]:
stopwords = ["тд","c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","специалист","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]

In [234]:
df['Чистые слова'] = df['Слова'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [256]:
df.drop("Раздел", axis=1, inplace=True)

In [238]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [250]:
# for i in df['Чистые слова'].iloc[1].split(" "):
#     p = morph.parse(i)[0]
#     print(p.normal_form)
    
a=[morph.parse(w)[0].normal_form for w in df['Чистые слова'].iloc[1].split(" ")]
print(a)

['назначение', 'использование', 'технический', 'обслуживание', 'уход', 'оборудование', 'безопасность', 'применение', 'основа', 'информатика', 'вычислительный', 'техника', 'метод', 'поиск', 'получение', 'соответствовать', 'информация', 'специальный', 'общий', 'характер', 'технический', 'условие', 'инструкция', 'внутренний', 'организационно-распорядительный', 'документ', 'экономический', 'субъект', 'регламентировать', 'способ', 'защита', 'персональный', 'данные']


In [251]:
df['Инф'] = df['Чистые слова'].apply(lambda x: " ".join([morph.parse(word)[0].normal_form for word in x.split()]))

In [253]:
data = df.copy()
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cvData = cv.fit_transform(data["Инф"]).toarray()

In [254]:
cvDF = pd.DataFrame(data=cvData, columns=cv.get_feature_names())